##### 1.Setup
Verify python3 or install it

In [ ]:
which python3 git jq

#if not installed for Linux
sudo apt-get install python3 
sudo apt-get install pip3
sudo apt-get install git
sudo apt-get install jq

#if not installed for Mac
brew install python3
brew install pip3
brew install git
brew install jq

#cloud
pip3 install dxpy
dx -h
git clone https://github.com/laderast/bash_bioinfo_scripts/


/home/dev105/mambaforge/bin/python3


##### Sources

In [ ]:

#tutorial
#https://github.com/raynamharris/Shell_Intro_for_Transcriptomics/blob/master/lessons/03_ReadWriteMoveCopy.mdwhoami, pwd, cd, ls, ls -tlh(sort by time, show details, human readable format)
#https://wikis.utexas.edu/display/bioiteam/Scott's+list+of+linux+one-liners
https://people.duke.edu/~ccc14/duke-hts-2018/cliburn/Bash_in_Jupyter.html 
https://laderast.github.io/bash_for_bioinformatics/
https://bioinformatics.ccr.cancer.gov/docs/b4b/Module1_Unix_Biowulf/Lesson1/
https://practicalcomputing.org/about.html
https://regex101.com/

Bash is a interpreter based on sh (shell)
locate database of path names


ls -tlh $PathDirectory
ls folder
ls -a #hidden files
ls -A #exclude current and parent directory
ls -l #characteristics
ls -lt #last modified
ls -R #recursive
ls -d */ #showing only directories
ls -l | grep -E '^d'
ls -l | grep -Ev '^d' #showing only files
ls *ipynb #globbing
ls *Text*
ls text? #only one character


full and relative paths /, ~~, ., .., cd, tab for autocompletion
\ split over multiple lines
with | a break is not necessary

# files
cat file1 file2 #explore at the same time
head -n 4 file
head -4 file
tail -n 10 file
tail +3 file
less file #view and scroll, q for quitting
cp file1 file2 $-R recursive
mkdir
mv #move or rename
rm #remove -r recursive, rmdir for empty, -rf including subdirectories
echo #print on the screen
> #output
>> #append output

#here doc to pass multiline EOF or END, 
#unquoted to use shell, - leading tab ignored "indentation"
uppercase $PWD or $(PWD)
cat << EOF > text.txt
crear dir
un documento
EOF

#touch file.txt creates an empty file


#nagivation
up down #previous next command
control+k delete from the cursor to the end
control+e move to the end
control+a move to the beginning
control+d delete the character under the cursor
control+c kill the current run

wc filename #word count -l lines -w words -c characters
* #wildcard
? #singlecharacter

gunzip SRR*_1R2.fastq.gz
sort -n file #sort by number, -r reverse, uniq (shortlist), uniq -c (counts)


grep #global regular expression print, lines
grep -w #limit to find words
grep -w "this is a pattern" file
grep -n #numbers the lines
grep -c #counts
grep -i #case insensitive
'^It' #starts the line with It
'[0-9]+' #line with numbers

grep -v #print lines that do not match
grep -o '[A-Z]*' #get rid of numbers
grep -E #no need to escape special characters, -Ev invert option
grep -r "hello" ./*txt #recursive
grep --color
grep -l #files
ls -d */ #directories

ls -l | grep -E '^d' #folders that start with d

## find
find . -name iris*
find . -name "*unix*ipynb"
find . -iname #case insensitive
find . -not -path "*ipynb_checkpoints/*" -iname "*unix*ipynb"
find . -name "*[csv|txt]" -maxdepth 1 #one depth 
-ctime +1 #1day ago 
-mtime -1 #modified <1d 
-mmin -15

#a set
[abc]
[a-z]
! #negates
[!a-d]*
[^A-Z] match anything not in the set
#pre-defined classes
['[:alpha:]'] #letters
['[:digit:]'] #numbers
['[:punct:]'] #comma or .

grep -E '()'
grep -E '[]'
grep -E ab+c # one or more of the preceding character abbbbbbcd
grep -E ab*cd # zero or more of the preceding character
grep -E 'ab{1,5}cd' #number of repetitions of the preceding character set
'\<the\>' # word boundaries
grep -E '([0-9]+).*\1'
'([0-9]+)_([0-9]+)_\1_\2' #repeated twice


. one character
+ one or more of the preceding character
* zero or more characters
^ start of the line
$ end of the line
[a|b|c] a or b or c
(cat|dog) cat or dog


grep "[2|4]" #lines 2 and 4
egrep

cat file.txt | awk -F '\t' '$1=="Mt"' | head -3
cat file.txt | awk -F '\t' '$1=="2"' {print $3} | sort | uniq -c
cat docs/echo.txt | awk 'BEGIN {FS = ","} ; $5 == "M"' > docs/male.txt

grep -A 1 ##will print an aditional line after matching

cut extracts columns separated by delimiters (-d)
cut -d " " -f 2 file.txt #column 2 or 1,3,5 or 1-5


sed for substitution
sed -n "1~2p" file.txt #-n not print, extract only odd numbered lines
sed -n '2000,2005p' #number of lines

s/pattern/replacement/g/
s->substitute
g globally or first pattern found
[0-9][0-9]* regular expression for matching a number 
\. escape
[[:space:]] matches a single space
sed "s/[0-9][0-9]*\.[[:space:]]//g" file.txt

sed to remove dupplicate characters
sed -E "s/(.)\1*/\1/g" junk.txt

convert DNA to RNA
cat file.txt | tr T U
#tr -d 'AT' (delete)
tr '\t' ','#tsv to csv
tr a-z A-Z
tr ACTG TGAC

convert reverse complement
grep -o '[ACTG]*' junk.txt | tr ACGT TGCA | rev
grep '[CG]' -o file | wc -l #count only CG

#find unique features
zcat $DATA_DIR/$GTF | grep -v '^#' | cut -f 3 | sort | uniq
#counts
zcat $DATA_DIR/$GTF | grep -v '^#' | cut -f 3 | sort | uniq -c
#first 3 exons
zcat $DATA_DIR/$GTF | grep -v '^#' | awk '$3 == "exon"' | head -3
#attributes
zcat $DATA_DIR/$GTF | grep -v '^#' | awk '$3 == "gene"' | cut -f 9 > genes.txt


extract odd lines
sed -n "1~2p" file.txt
zcat $DATA_DIR/$FASTQ | sed -n '2~4p;' | head | tr -d '\n' #extract every 4th line

uniq #recognize two or more adjacent lines
uniq -c #how many lines are repeated 
uniq -w N #first N characters of every line to decide, necessary to sort first

history | grep "pwd" #search in your history

\ continue to next line or pipe |

cd (cd - previous directory)
mkdir -p seqs/data (create a parent directory and intermediate directories, rmdir -p also to remove them)
efetch -db=nuccore -format=fasta -id=CM000046.1 > seqs/CM000046.1.fa


gzip #to compress
gunzip #to uncompress
ls *fa | xargs -n1 bgzip 
zcat file.fa.gz | head
tar #to compress directories (-cvzf or -xvzf to uncompress and recover)
	c compress
	x extract
	z gzip or gunzip
	f files
	v verbose
	t view list

#integrity for data transfer checksum
md5sum *fa > MD5_checksum
#to check
md5sum -c MD5_checksum
#to compare then with the original file
diff -u file1 file2 | cat

set variables
dirpath='PATH'
fasta='file'
echo ${dirpath} #use always ${}
variables=$(pwd) #without spaces for commands
echo "hello ${NAME1}"
((NUM++)) double parantheses for mathematical expressions
#seq function
generates a range of numbers
seq 3
seq 2 5
seq 5 2 9

#if to check for file existence
if [-f hello.txt]; then
	cat hello.txt
else
	echo "no such file"
fi

if [! -f "data/file"]; then
	wget url -O data/file
fi

#test for conditional evaluation
test -f file o [-f file]

#Loops
for i in SRRR*;
do
    echo ${i} #inserted in a line with other characters
done
for file in copy-SRR534005*; 
echo $file ${file//SRR534005/chicken}; 
do mv $file ${file//SRR534005/chicken}; 
done

for FILE in $(ls *ipynb); do
    echo $FILE
done

for ((i=0; i<5; i++)); do
    echo $i
done

test -a = &&
test -o = ||
=~ to match regular expressions patterns

for FILE in $(ls); do
    if [[ "${FILE}" =~ ^Bash.*sh ]]; then
        echo $FILE
    fi
done

COUNTER=10
while [ $COUNTER -gt 0 ]; do
    echo $COUNTER
    COUNTER=$(($COUNTER - 1))
done

-lt #less than
-gt #greater than
== #equaity
!= #inequality

#braces in loops
for NUM in {000..005}; do
    echo mkdir EXPT-${NUM}
done
echo foo.{c,cpp,h,hp}
#loop first 10 records
LINE_NUM=0
while read LINE
do
    if [[ $LINE_NUM -lt 10 ]]; then
        echo $LINE
        (( LINE_NUM++ ))
    fi
done < genes.txt

#command expressions $(command), #basename without path, dirname
#remove the file extension Y=${x%.*}
for FILE in $(ls ~/docs/*)
do
    DIR_NAME=$(dirname $FILE)
    FILE_NAME=$(basename $FILE)
    NAME=${FILE_NAME%.*}
    NEW_DIR=$DIR_NAME/$NAME
    NEW_FILE=${NAME}-copy.txt
    mkdir -p $NEW_DIR
    cat $FILE | head -3 | tail -2  > $NEW_DIR/$NEW_FILE
done



for FILE in $(ls $DATA_DIR/*_MA_J_S20_*gz)
do
    FIRST=$(zcat $FILE 2>/dev/null | head -1)
    echo $(basename $FILE)
    echo $FIRST
    echo
done
#script
#!/path/to/shell or bash path/file
#!/bin/bash or which bash

chmod +x /path/to/script

#!/bin/bash
# "#" gives you the number from the command line
echo $# 
for ARG in {0..$#}; do
    echo $ARG
done
EOF


bash echo.sh
for file in *.fastq
do
	echo ${file}
done

#script with input specifications
bash wordcount.sh *R1.fastq
for file in ${@}
do
	wc ${file}
done
#structure
bin, data, results, manuscript


cat > count_lines.sh << 'EOF'
#!/bin/bash

for FILE in $(ls)
do
    wc -l "${FILE}"
done
EOF

#### Loops

  *  `for` is use for iterating over a fixed number of items (may be unknown at time of coding).
  *  `while` is use for iterating until a certain condition is met

#### Conditionals (for flow control)


In [ ]:
#### Loops
##### Examples of for
for file in ../data/${@}
do
	wc ${file} >> ../data/wordcount.txt
done

for file in ../data/${@}
do
	output=$(basename ${file} .fastq)-wordcount.txt 
	wc ${file} >> ../results/${output}
done

for loops
for i in $( ls ); do echo item: $i 
done

##### Examples of while
while true; do echo ‘hello’ done

#### Conditionals
if [ "foo" = "foo" ]; then
else
fi




Sources for scripts
https://jeroenjanssens.com/dsatcl/
https://missing.csail.mit.edu/2020/course-shell/
https://swcarpentry.github.io/shell-novice/04-pipefilter/index.html https://datascienceatthecommandline.com/2e/chapter-2-getting-started.html?q=stdin#combining-command-line-tools

In [ ]:
##### Iterating with xargs -I one by one -0 avoid blanks
ls *.fasta | xargs -I% sh -c 'head %; echo "\n---\n"'
#sh -c "to call shell instead of bash"
#https://www.baeldung.com/linux/xargs-multiple-arguments
my_variable="ggplot2"

echo "My favorite R package is ${my_variable}"
my_var="chr1"
echo "${my_var}_1.vcf.gz"

In [ ]:

#script

#STDIN -> STDOUT+STDERR
I/O redirections: |,<,>
ps -aef #show a list of all processes running
bash sam_run my_file.bam out_stats.txt
#!/bin/bash/
samtools stats $1 > $2
#!/bin/bash/
samtools stats $2 > $1 #output primero

In [ ]:
#to hack SIGPIPE error in Jupyter
cleanup () {
    :
}

trap "cleanup" SIGPIPE
#safety
set -u #unset variables as an error and exit

In [ ]:
https://github.com/raynamharris/Shell_Intro_for_Transcriptomics/tree/master
https://practicalcomputing.org/
https://explainshell.com/
A FASTQ file normally uses four lines per sequence.

Line 1 begins with a '@' character and is followed by a series of information that uniquely identifies each read. Thus, this is our sequence. Here are two example:

@HWI-ST1097:127:C0W5VACXX:5:1101:4820:2124 1:N:0:CTCAGA
@SRR534005.299 HWI-ST945:93:c02g4acxx:3:1101:16012:2225/1

Line 2 is the raw sequence. The lenght of this sequence should correspond to the length of the read requested (i.e. 75 bp, 150 bp). These are AGTC or N uppercase characters.

Line 3 is always '+' for output from the Illumina machines in the Genomic Sequencing and Analysis Facility.

Line 4 is a string of Ascii-encoded base quality scores encodes the quality values for the sequence in Line 2. This string contain the same number of symbols as letters in the sequence on Line 2.
UNIX es un sistema operativo desarrollado en 1969. Se caracteriza principalmente por:
Advertencia Utilizare terminos sin traducir por que considero que son importantes para seguir entendiendo adecuadamente Linux.
UNIX filosofia
    - El uso de texto plano para almacenamiento.
    - Sistema jerarquico de archivos del sistema.
    - Los dispositivos e IPC son tratados como archivos.
    - Multiples programas pueden ser ejecutados juntos usando "pipes" en el interprete (a partir de ahora command line).
Terminal Dispositivo usado para enviar y recibir datos del computador.
Terminal.app Es un emulados del sistema operativo que permite interactuar con la computadora mediante el command line.
Shell Es el programa que permite al sistema UNIX entender los comandos. *Usaremos bash, the bourne again shell*


Commandos sencillos
pwd
ls
cd
cp
mv
mkdir
rmman
cat/zcat
echo

wildcards o caracteres propios de shell
* matchs all
? matchs a single character
[A-Z]
/
./
../
~

absolute path
cd /usr/local/bin
relative path
cd ../lib


Practica
cd
pwd
ls -tl
mkdir -p test-dir/lab1
cd test-dir/lab1
mkdir temp1 temp2 temp3
mv temp1 temp2 temp3
rm -rf temp*
cd ..
cd -
pwd


Brace expansion
cho dog-{gone,bowl,bark}
dog-gone dog-bowl dog-bark

 mkdir -p zmays-snps/{data/seqs,scripts,analysis}
-p crear todos los subdirectorios necesarios
Shell tiene ademas las funciones de autocompletar e historia
la tecla tab (completara el comando o el nombre del archivo)
Para acceder a la historia command se puede usar las teclas arriba o abajo.
standard out to a file > o >> (>> agregara)

por ejemplo concatenar fasta files
cat *.fa > list.fa

crear un log de errores
command &> logfile
command &>> logfile

2> for errors

tail-f (redirigir el output a la pantalla)

input redirection <
-single dash argument (standard input)

Unix Pipes
redirigir outputs entre programas
El resultado de el programa X sera considerado input del programa Y
Imagina descargar y analizar un archivo usando el metodo clasico de windows, tendria que crear muchos pasos e interactuar en todos ellos. Usando Unix puedo escribir todo y automatizarlo en un solo codigo.

grep -v "^>" ../data/tb1.fasta | grep --color -i "[^ATCG]"
CCCCAAAGACGGACCAATCCAGCAGCTTCTACTGCTAYCCATGCTCCCCTCCCTTCGCCGCCGCCGACGC


he Unix program tee diverts a copy of your pipeline’s standard output stream to an intermediate file while still passing it through its standard output:
program1 input.txt | tee intermediate-file.txt | program2 > results.txt

We can tell the Unix shell to run a program in the background by appending an ampersand (&) to the end of our command. For example:

program1 input.txt > results.txt &
control + Z 
enviarlo a background bg
shell muestra el numero ID o PID
jobs
traerlo de vuelta fg
fg %<numero>

aniquilar el proceso
Control + C

kill -9 <PID>
kill -9 %<num> numero de trabajo


Unix programs exit with an exit status, which indicates whether a program terminated without a problem or with an error. By Unix standards, an exit status of 0 indicates the process ran successfully, and any nonzero status indicates some sort of error has occurred (and hopefully the program prints an understandable error message, too). The exit status isn’t printed to the terminal, but your shell will set its value to a shell variable named $?. We can use the echo command to look at this variable’s value after running a command:
program1 input.txt > results.txt; echo $?
The shell provides two operators that implement this: one operator that runs the subsequent command only if the first command completed successfully (&&), and one operator that runs the next command only if the first completed unsuccessfully (||).

For example, the sequence program1 input.txt > intermediate-results.txt && program2 intermediate-results.txt > results.txt will execute the second command only if previous commands have completed with a nonzero exit status.
program1 input.txt > intermediate-results.txt || echo "warning: an error occurred" will print the message if error has occurred.
True
echo $?
0
false
echo $?
1
true && echo "works"
works
true || echo "works"  (no saldra nada, solo si falla)
false && echo "first command was a success"

&& solo avisa cuando se cumple
|| solo avisa cuando no se cumple

command substitution
echo "There are $(grep -c '^>' ../data/improper.fa) entries in my FASTA file."
There are 2 entries in my FASTA file.

mkdir results-$(date +%F) #or
mkdir results-`date +%F`

RMarkdown
https://data-skills.github.io/unix-and-bash/01-unix-git-intro/index.html

# Shell
command line que te permite controlar la computadora desde el teclado.
reproducible, eficiente y rapido.
Trabajar con cloud computers

# whoami
# pwd
# ls
ls -t #ordenado por tiempo
ls -l #detalles
ls -h #tamano

# cd 
change directory
 relative path
 full path
 root directory
 home directory
 . este directorio
 .. un directorio arriba
 ../.. dos directorios arriba
 ~ home directorio
 cd #sin nada -> home directory

# head y tail #
muestra 10 primeras lineas
head -n 4
tail -n 4
# less #
muestra y navega en el contenido del archivo
usar q para salir

# cp #
cp origen destino
mv mover o renombrar

# rm #
eliminar
-r directorios
rmdir directorios

# echo #
mostrar en la pantalla
echo 'hola!'
echo -e "3 Hello\n4 again" >> hello.txt

2> #output and stderr

# echo > y >> 
>> se agrega al archivo existente

# cat
muestra el contenido, pero tambien concatena varios archivos

# Navegacion #

control + d : eliminar el caracter en el cursor
control + a : al inicio de la linea
control + e : al final de la linea
arriba : comando previo
abajo : siguiente comando
control + k : elimina desde el cursor al final de la linea
control + c : forzar cerrar una tarea (salir)






# wc #
Significa: "word count"
** Uso: cuenta lineas, palabras o caracteres. **
wc -l #numero de lineas
wc -w #numero de palabras
wc -c #numero de caracteres

wc -l *.fastq > lenghts.txt

sort -n lenghts.txt #ordena de forma numerica, -t',' -k 1 first column separated by commas
head -n lenghts.txt
wc *.fastq | sort -n | head -1

# * #
wildcard 0 o mas caracteres
# ? #
wildcard 1 caracter

# gunzip #
unzip

https://wikis.utexas.edu/display/bioiteam/Scott's+list+of+linux+one-liners
## grep ##
Significa literalmente: “global/regular expression/print”

**Uso: busca patrones (palabras o caracteres)y muestra las lineas que coinciden.**

grep [argumento] [patron] [archivo]
grep fastq lista.txt
grep -w day haiku.txt (word or pattern "")
grep -n it haiku.txt (numbers the lines)
grep -i case insensitive
grep -v do not match
grep -A 1 also prints one line after

uniq recognize two or more adjacent lines are the same
uniq -c para contar
uniq -w N solo usa los xxx caracteres iniciales para evaluar si son iguales
sort haiku.txt | uniq -c 

history historial
history | grep "pwd"

# Bucles (Loops) #
for -> repite el/los comandos para cada uno de los elementos de una lista

for i in *.fastq
do
ls $i
done

$i -> variable
do -> le dice a shell que daremos comandos
ls -> comando o echo
done -> fin

$ for file in *.fastq
> do
>    echo " fastq $file"
> done

Nota: comilla y doble comilla -> doble comilla invocara un comando y reemplazara la variable con el archivo

Nota: echo ${variable}
echo 'esto!'
echo esto

Por ejemplo, si se necesita todos los R1
$ ls *R1*

$ for R1 in *R1*
> do
>    echo $R1
> done

Ahora obtener la R2
newvariablename=${oldvariablename//oldtext/newtext}

$ for R1 in *R1*
> do
>    R2=${R1//R1_001_small.fastq/R2_001_small.fastq}
>    echo $R1 $R2
> done

$ for R1 in *R1*
> do
>    R2=${R1//R1_001_small.fastq/R2_001_small.fastq}
>    echo "do something to $R1 and also to $R2
> done

for R1 in *R1*
do
   R2=${R1//R1_001_small.fastq/R2_001_small.fastq}
   R1paired=${R1//.fastq/_paired.fastq.gz}
   R1unpaired=${R1//.fastq/_unpaired.fastq.gz}	
   R2paired=${R2//.fastq/_paired.fastq.gz}
   R2unpaired=${R2//.fastq/_unpaired.fastq.gz}	
   echo "java -jar $WORK/IntMolModule/STG/practiceData/Trimmomatic-0.36 PE -threads 32 -phred33 $R1 $R2 $R1paired $R1unpaired $R2paired $R2unpaired"
done


#save it as cmds

rm trimmomatic.cmds
for R1 in *R1*
do
   R2=${R1//R1_001_small.fastq/R2_001_small.fastq}
   R1paired=${R1//.fastq/_paired.fastq.gz}
   R1unpaired=${R1//.fastq/_unpaired.fastq.gz}	
   R2paired=${R2//.fastq/_paired.fastq.gz}
   R2unpaired=${R2//.fastq/_unpaired.fastq.gz}	
   echo "java -jar $WORK/IntMolModule/STG/practiceData/Trimmomatic-0.36/trimmomatic-0.36.jar PE -threads 32 -phred33 $R1 $R2 $R1paired $R1unpaired $R2paired $R2unpaired" >> trimmomatic.cmds
done
cat trimmomatic.cmds
## Bash Scripts 
extension .sh
son archivos ejecutables
se ejecutan como bash archivo.sh


for file in ${@}
do 
    wc ${file}
done

Organizacion de proyectos
bin
data
results
manuscripts


for file in ../data/${@}
do
    output=$(basename ${file} .fastq)-wordcount.txt
    wc ${file} >> ../data/wordcount.txt
done

mkdir dataseq

while read
fasterq-dump -p --split-files $inputsra -O ./dataseq



echo "Start!"

for sra in (txt)
do
    echo "${p}"
    while [ $(jobs | wc -l) -ge "$maxproc" ]
    do
        sleep 1
    done
    base=${fname%_R1*}
    echo starting new job with ongoing=$(jobs | wc -l)
    bwa-0.7.5a/bwa mem -t 4 human_g1k_v37.fasta "${base}_R1.fastq.gz" "${base}_R2.fastq.gz" >"$destdir/${base}_R1_R2.sam" &
done


#gunzip

Compressing multiple files with gzip
To compress multiple files with gzip, you can use *.txt wildcard to select all files with .txt extension in current directory.
gzip *.txt
This command will compress all files with .txt extension in current directory, creating new compressed files with .gz extension.
Compressing files while keeping original file
To compress a file with gzip while keeping original file, you can use -k option.
gzip -k file.txt

From <https://www.tutorialspoint.com/using-gzip-and-gunzip-in-linux> 

wget http://kandurilab.org/users/santhilal/courses/UNIX/materials/course_file_repo.zip

Workspace: shell or shell prompt
Linux - Ubuntu terminal - bash



#list
ls 
#detailed list (long list)
ls -l 
ls -lh
#pwd
#mkdir
#cp
#mv (risky)
#nano will create files
Alt+ change between files
Control o to save
Control X exit
sudo apt-get update


 'file1.txt' -> 'myfile1.txt', 'file2.txt' -> 'myfile2.txt'
Now let's merge the created text files using command 'cat' (concatenate).

$$ cat myfile1 myfile2 > mergedfile.txt # This merges the contents of the file in a new file
$$ cat mergedfile.txt # Display the content of merged file

Rm myfile*
Rm -mydummy
Sudo apt-get install unzip

Unzip file
Head
Less
More

Salir con q


Count words or lines
Wc -l *.list (list es el tipo de nombre)


Match genes start with 'USP'.
$$ cat H0.list | grep "^USP*" # Here '^' means 'line starts with'
$$ cat H0.list | grep "USP*"
Match genes ending with '.' followed by some digit/number.
$$ cat H0.list | grep "\.[0-9]$"
\. -> to scape of matching
* suffix
. Any digit
$ the ending part
Beginning of line ( ^ )
Count of empty lines 
"^$"  FILE
Grep -w "…." (only 4 characters)
*. 
\+ one or more ocurrences (sin considerar espacios) dentro de la oracion mas de un espacio
Cero o no espacios *
\? Cero o una ocurrencia
\ escape the special character
Character class [0-9]

Exception in the character class grep -I "^[^aeiou]"

Grep -B (lineas antes y despues)





$$ cat hg38_gene_annotation_mart_export.txt | grep -v "^#" | cut -f1 | sort | uniq | wc -l # This counts total number of unique genes in the annotation
$$ cat hg38_gene_annotation_mart_export.txt | grep -v "^#" | cut -f7 | sort | uniq | wc -l # This counts total number of unique transcripts
$$ cat hg38_gene_annotation_mart_export.txt | grep -v "^#" | grep -w "protein_coding" | cut -f1 | sort | uniq | wc -l # Counts only total number of protein coding genes

Zcat to open .gz files

zcat Homo_sapiens.GRCh38.87_badStructure.table.gz | sed 's/gene_id "//' | sed 's/gene_name "//' | sed 's/gene_biotype "//' | sed 's/"//g' | sed '1i\Geneid\tGeneSymbol\tChromosome\tClass\tStrand\tLength' | sed 's/ //g' | head

SED
https://www.gnu.org/software/sed/manual/sed.html

Sed is more efficient and make only one pass over the input (filter text in a pipeline)
Sed $script INPUTFILE
sed 's/hello/world/' input.txt > output.txt

Alternatives:
sed 's/hello/world/' input.txt > output.txt
sed 's/hello/world/' < input.txt > output.txt
cat input.txt | sed 's/hello/world/' - > output.txt

Sobre el mismo archivo 
-i
sed -i 's/hello/world/' input.txt #sobre el mismo archivo
#otros: w s///w

Use -n to suppress output, and the p command to print specific lines. The following command prints only line 45 of the input file:

sed -n '45p' file.txt
sed treats multiple input files as one long stream. The following example prints the first line of the first file (one.txt) and the last line of the last file (three.txt). Use -s to reverse this behavior.

sed -n  '1p ; $p' one.txt two.txt three.txt

Without -e or -f options, sed uses the first non-option parameter as the script, and the following non-option parameters as input files. If -e or -f options are used to specify a script, all non-option parameters are taken as input files. Options -e and -f can be combined, and can appear multiple times (in which case the final effective script will be concatenation of all the individual scripts).

The following examples are equivalent:

sed 's/hello/world/' input.txt > output.txt

sed -e 's/hello/world/' input.txt > output.txt
sed --expression='s/hello/world/' input.txt > output.txt

echo 's/hello/world/' > myscript.sed
sed -f myscript.sed input.txt > output.txt
sed --file=myscript.sed input.txt > output.txt
The full format for invoking sed is:

sed OPTIONS... [SCRIPT] [INPUTFILE...]
sed may be invoked with the following command-line options:

--version
Print out the version of sed that is being run and a copyright notice, then exit.

--help
Print a usage message briefly summarizing these command-line options and the bug-reporting address, then exit.

-n
--quiet
--silent
By default, sed prints out the pattern space at the end of each cycle through the script (see How sed works). These options disable this automatic printing, and sed only produces output when explicitly told to via the p command.

--debug
Print the input sed program in canonical form, and annotate program execution.

$ echo 1 | sed '\%1%s21232'
3

$ echo 1 | sed --debug '\%1%s21232'
SED PROGRAM:
  /1/ s/1/3/
INPUT:   'STDIN' line 1
PATTERN: 1
COMMAND: /1/ s/1/3/
PATTERN: 3
END-OF-CYCLE:
3
-e script
--expression=script
Add the commands in script to the set of commands to be run while processing the input.

-f script-file
--file=script-file
Add the commands contained in the file script-file to the set of commands to be run while processing the input.

-i[SUFFIX]
--in-place[=SUFFIX]
This option specifies that files are to be edited in-place. GNU sed does this by creating a temporary file and sending output to this file rather than to the standard output.1.

This option implies -s.

When the end of the file is reached, the temporary file is renamed to the output file’s original name. The extension, if supplied, is used to modify the name of the old file before renaming the temporary file, thereby making a backup copy2).

This rule is followed: if the extension doesn’t contain a *, then it is appended to the end of the current filename as a suffix; if the extension does contain one or more * characters, then each asterisk is replaced with the current filename. This allows you to add a prefix to the backup file, instead of (or in addition to) a suffix, or even to place backup copies of the original files into another directory (provided the directory already exists).

If no extension is supplied, the original file is overwritten without making a backup.

Because -i takes an optional argument, it should not be followed by other short options:

sed -Ei '...' FILE
Same as -E -i with no backup suffix - FILE will be edited in-place without creating a backup.

sed -iE '...' FILE
This is equivalent to --in-place=E, creating FILEE as backup of FILE

Be cautious of using -n with -i: the former disables automatic printing of lines and the latter changes the file in-place without a backup. Used carelessly (and without an explicit p command), the output file will be empty:

# WRONG USAGE: 'FILE' will be truncated.
sed -ni 's/foo/bar/' FILE
-l N
--line-length=N
Specify the default line-wrap length for the l command. A length of 0 (zero) means to never wrap long lines. If not specified, it is taken to be 70.

--posix
GNU sed includes several extensions to POSIX sed. In order to simplify writing portable scripts, this option disables all the extensions that this manual documents, including additional commands. Most of the extensions accept sed programs that are outside the syntax mandated by POSIX, but some of them (such as the behavior of the N command described in Reporting Bugs) actually violate the standard. If you want to disable only the latter kind of extension, you can set the POSIXLY_CORRECT variable to a non-empty value.

-b
--binary
This option is available on every platform, but is only effective where the operating system makes a distinction between text files and binary files. When such a distinction is made—as is the case for MS-DOS, Windows, Cygwin—text files are composed of lines separated by a carriage return and a line feed character, and sed does not see the ending CR. When this option is specified, sed will open input files in binary mode, thus not requesting this special processing and considering lines to end at a line feed.

--follow-symlinks
This option is available only on platforms that support symbolic links and has an effect only if option -i is specified. In this case, if the file that is specified on the command line is a symbolic link, sed will follow the link and edit the ultimate destination of the link. The default behavior is to break the symbolic link, so that the link destination will not be modified.

-E
-r
--regexp-extended
Use extended regular expressions rather than basic regular expressions. Extended regexps are those that egrep accepts; they can be clearer because they usually have fewer backslashes. Historically this was a GNU extension, but the -E extension has since been added to the POSIX standard (http://austingroupbugs.net/view.php?id=528), so use -E for portability. GNU sed has accepted -E as an undocumented option for years, and *BSD seds have accepted -E for years as well, but scripts that use -E might not port to other older systems. See Extended regular expressions.

-s
--separate
By default, sed will consider the files specified on the command line as a single continuous long stream. This GNU sed extension allows the user to consider them as separate files: range addresses (such as ‘/abc/,/def/’) are not allowed to span several files, line numbers are relative to the start of each file, $ refers to the last line of each file, and files invoked from the R commands are rewound at the start of each file.

--sandbox
In sandbox mode, e/w/r commands are rejected - programs containing them will be aborted without being run. Sandbox mode ensures sed operates only on the input files designated on the command line, and cannot run external programs.

-u
--unbuffered
Buffer both input and output as minimally as practical. (This is particularly useful if the input is coming from the likes of ‘tail -f’, and you wish to see the transformed output as soon as possible.)

-z
--null-data
--zero-terminated
Treat the input as a set of lines, each terminated by a zero byte (the ASCII ‘NUL’ character) instead of a newline. This option can be used with commands like ‘sort -z’ and ‘find -print0’ to process arbitrary file names.

If no -e, -f, --expression, or --file options are given on the command-line, then the first non-option argument on the command line is taken to be the script to be executed.

If any command-line parameters remain after processing the above, these parameters are interpreted as the names of input files to be processed. A file name of ‘-’ refers to the standard input stream. The standard input will be processed if no file names are specified.

For printing specific range of lines from the file.

$$ sed -n 1,2p Homo_sapiens.GRCh38.87_annotation.table
$$ sed -n 4,6p Homo_sapiens.GRCh38.87_annotation.table
For deleting lines

$$ cat Homo_sapiens.GRCh38.87_annotation.table | sed '1d' | head

Translate or replace
Tr https://www.thegeekstuff.com/2012/12/linux-tr-command/
$$ echo "santhilal" | tr a-z A-Z



https://www.gnu.org/software/gawk/manual/html_node/Comparison-Operators.html

https://genome.ucsc.edu/FAQ/FAQformat#format1


pipe: combining different commands to get required output

sudo apt-get upgrade -y

sudo apt-get dist-upgrade -y

sudo apt-get autoremove -y

sudo apt-get autoclean -y


Onedrive
wget -qO - https://download.opensuse.org/repositories/home:/npreining:/debian-ubuntu-onedrive/xUbuntu_22.04/Release.key | gpg --dearmor | sudo tee /usr/share/keyrings/obs-onedrive.gpg > /dev/null

echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/obs-onedrive.gpg] https://download.opensuse.org/repositories/home:/npreining:/debian-ubuntu-onedrive/xUbuntu_22.04/./" | sudo tee /etc/apt/sources.list.d/onedrive.list

sudo apt-get update

sudo apt install --no-install-recommends --no-install-suggests onedrive


#to get a detailed list of folders
Ls -shl *


Linux
Turn windows features on or off
check windows subsystem for linux
follow the instructions and reboot
microsoft store → ubuntu
wsl --install
Run a script
	1. Write the script file using
nano script-name-here.sh
	2. Set execute permission on your script using chmod command :
chmod +x script-name-here.sh
	3. To run your script :./script-name-here.shAnother option is as follows to execute shell script:sh script-name-here.shORbash script-name-here.sh
“terminal”, “command”, “prompt” or “shell” or Ctrl-Alt-T.
prompt, and you might sometimes see instructions that say “bring up a prompt”, “open a command prompt”, “at the bash prompt”
pwd is an abbreviation of ‘print working directory’ (show the path)
change directories (cd), another to list their contents (ls), a third to rename or move files (mv)
cd /
pwd
cd home
pwd
cd ..
pwd
cd
pwd
cd ../.. #if you have to move up through multiple levels
pwd
cd ../../etc
pwd
cd /etc #absolute pathway
pwd
whoami
cd /home/USERNAME/Desktop
pwd
#“starting from the root directory”
cd ~
pwd

cd ~/Desktop
pwd
https://ubuntu.com/tutorials/command-line-for-beginners#4-creating-folders-and-files
Mkdir - create files
mkdir /tmp/tutorial
cd /tmp/tutorial
mkdir dir1 dir2 dir3
mkdir -p dir4/dir5/dir6
ls
-p or --parent
mkdir "folder 1"
mkdir 'folder 2'
mkdir folder\\ 3
mkdir "folder 4" "folder 5"
mkdir -p "folder 6"/"folder 7"
ls
ls > output.txt
#we can use the cat command to look at its content:
cat output.txt
echo "This is a test"
echo "This is a test" > test_1.txt
echo "This is a second test" > test_2.txt
echo "This is a third test" > test_3.txt
ls
cat test_1.txt test_2.txt test_3.txt
question mark (”?”) can be used to indicate “any single character” within the file name. An asterisk (”*”) can be used to indicate “zero or more characters”. These are sometimes referred to as “wildcard” characters. A couple of examples might help, the following commands all do the same thing:

cat test_1.txt test_2.txt test_3.txt
cat test_?.txt
cat test_*
cat t*, meaning “concatenate all the files whose names start with a t and are followed by zero or more other characters”. Let’s use this capability to join all our files together into a single new file, then view it:

cat t* > combined.txt
cat combined.txt
Up Arrow and Down Arrow keys to move back and forth through the history of commands you’ve used. Press the Up Arrow a couple of times to get to the first cat and press Enter to run it, then do the same again to get to the second.

If you do want to append to, rather than replace, the content of the files, double up on the greater-than character:
cat t* >> combined.txt
echo "I've appended a line!" >> combined.txt
cat combined.txt
#to see more text
less combined.txt 
Clear - clear the windows
Mv “filepath” newname
mv combined.txt dir1
mv dir1/* .
mv combined.txt ..
mv combined.txt test_* dir3 dir2 ls ls dir2
mv dir2/combined.txt dir4/dir5/dir6
ls dir2
ls dir4/dir5/dir6
cp dir4/dir5/dir6/combined.txt .
ls dir4/dir5/dir6
ls
cp combined.txt backup_combined.txt
ls
mv backup_combined.txt combined_backup.txt
ls
mv "folder 1" folder_1
mv "folder 2" folder_2
mv "folder 3" folder_3
mv "folder 4" folder_4
mv "folder 5" folder_5
mv "folder 6" folder_6
ls
Chmod +x filename - make executable
head .bashrc - go to the bash
Cp .bashrc bashrc.bak
Nano .bashrc
Write at the end:
export PATH=$PATH:”filepath”
Rm - remove
rm dir4/dir5/dir6/combined.txt combined_backup.txt
Perhaps we should remove some of those excess directories as well:
rmdir folder_*
rmdir -p dir1/dir2/dir3
rm -r folder_6
Rm -rf director (remove not empty directory)
ls
If you’re at all uncertain use the -i (interactive) option to rm, which will prompt you to confirm the deletion of each file; enter Y to delete it, N to keep it, and press Ctrl-C to stop the operation entirely.

How many lines are there in your combined.txt file? The wc (word count) command can tell us that, using the -l switch to tell it we only want the line count (it can also do character counts and, as the name suggests, word counts):
wc -l combined.txt
how many files and folders are in your home directory, and then tidy up after yourself, you could do this:
ls ~ > file_list.txt
wc -l file_list.txt
rm file_list.txt
piping the data from one command to another
ls ~ | wc -l
t’s better to use less to view it, and that advice still applies when using a pipe (remember, press q to quit):

ls /etc | less
tarUnix has a command, uniq, that will only output unique lines in the file.
cat combined.txt | uniq | wc -l
cat combined.txt | uniq | less

su command. This is variously described as being short for ‘superuser’ or ‘switch user’
logout command (or Ctrl-D shortcut)
cat /etc/shadow
sudo cat /etc/shadow
apt or apt-get commands. If the instructions require you to first add a new software repository to your system, using the apt-add-repository command, by editing files in /etc/apt
Shortcut to open the terminal in Ubuntu:
	Control+Alt+T
Update Ubuntu and then install a software
sudo apt update
sudo apt install evolution
sudo apt install evolution-ews
tree
cd /tmp/tutorial
tree
sudo su, be aware that every command after that will be running as the root user.
dot (”.”) is enough to make it disappear.

cd /tmp/tutorial
ls
mv combined.txt .combined.txt
ls
to continue working with them
cat .combined.txt
mkdir .hidden
mv .combined.txt .hidden
less .hidden/.combined.txt
can use the -a (show all) switch to ls to make it show everything in a directory, including the hidden files and folders:
ls
ls -a
ls .hidden
ls -a .hidden
As for our recently installed tree command, that works in a similar way (except without an appearance by . and ..):
tree
tree -a
rm -r /tmp/tutorial
ls /tmp
As a last step, let’s close the terminal. You can just close the window, but it’s better practice to log out of the shell. You can either use the logout command, or the Ctrl-D keyboard shortcut. If you plan to use the terminal a lot, memorising Ctrl-Alt-T to launch the terminal and Ctrl-D to close it will soon make it feel like a handy assistant that you can call on instantly, and dismiss just as easily

<http://linuxcommand.org/tlcl.php>
curl, wget, pip, npm, make,
Evolution is the new software
#To know the IP server dig +short myip.opendns.com @resolver1.opendns.com
https://xie186.github.io/Novice2Expert4Bioinformatics/install-bioinformatics-software-in-linux.html
https://hackr.io/blog/basic-linux-commands
https://youtu.be/7-G_dQr7B44
Install MUSCLE using binaries:
https://youtu.be/UJx34KVLIgI
Install PhyML using binaries:
https://github.com/vappiah/bioinfoscr...
Gene extractor script is available here
https://itol.embl.de/
iTOL browser
https://youtu.be/2tMQYi_12IQ
Python codes explained
Installing muscle
https://www.drive5.com/muscle/downloads.htm
Select Linux 64x copy link
Then control alt T to open the terminal
To download use: wget “link”
To execute: ar xvfz “filepath”
Installing RAxML
Copy the zip link
CLIPKIT - trimming
https://jlsteenwyk.com/ClipKIT/advanced/index.html
https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.3001007
Mesquite
cd bin
/bin$ cd MrBayes
cd src ./mb

#move a list of files into a new folder
mv-t dest_dir $(< text.file)


Estructura 
Command [options]
Ejemplos:
Ls /path/
Pwd
Cat concatenate
Cd change directory
• cut - Cut: Choose a column from a tab-delimited file (spreadsheet)
• Example: cut -f (column number to choose) file.txt
• head - Head: See the first few lines of a file
• Example: head -n (number of lines to show) file.txt
• ls - List: Show the content of a directory
• Example: ls /directory/to/show
• mkdir - Make directory: Make a new directory/folder
• Example: mkdir name_of_new_directory
• mv - Move: Move a file(s) or directory
• Example: mv /file/at/old/location/ /new/location/to/move/to
• rm - Remove: Delete a file/directory (must use -r option to delete a directory)
• Example: rm name_of_file_to_delete
#To download 
wget http://address
-o redirects to path
-o- redirects to standard output for piping
-q supresses messages

From <https://cyverse-leptin-rna-seq-lesson-dev.readthedocs-hosted.com/en/latest/section-3.html> 

special characters (e.g. !#$%^&* etc.). Also, you cannot use spaces in file names

From <https://cyverse-leptin-rna-seq-lesson-dev.readthedocs-hosted.com/en/latest/section-3.html> 

https://cyverse-leptin-rna-seq-lesson-dev.readthedocs-hosted.com/en/latest/section-3.html
https://www.freecodecamp.org/espanol/news/grep-command-tutorial-how-to-search-for-a-file-in-linux-and-unix-with-recursive-find/
https://tldp.org/LDP/Bash-Beginners-Guide/html/sect_09_02.html
https://datacarpentry.org/wrangling-genomics/
https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands
https://sanbomics.com/2022/01/08/complete-rnaseq-alignment-guide-from-fastq-to-count-table/
RNAseq tutorial - part 1 - building STAR genome index
Virus genome assembly and consensus sequence from reads
https://cyverse-leptin-rna-seq-lesson-dev.readthedocs-hosted.com/en/latest/section-8.html




https://gist.github.com/darencard/e1933e544c9c96234e86d8cbccc709e0

https://www.thegeekstuff.com/2010/03/awk-arrays-explained-with-5-practical-examples/
https://notebook.community/crazyhottommy/scripts-general-use/Shell/Awk_anotates_vcf_with_bed


In [ ]:
whoami, pwd, ls, ls -l


sudo apt install openjdk-17-jdk
https://gist.github.com/DannyArends/04d87f5590090dfe0dc6b42e5e1bbe15
RNA Sequencing - Setup and Prerequisites

https://www.youtube.com/watch?v=PlqDQBl22DI&t=14s

How to manipulate gene expression data from NCBI GEO in R using dplyr | Bioinformatics for beginners



https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE19080



Stem-like exhausted and memory CD8+ T cells in cancer
In vivo clinical molecular imaging of T cell activity


Sudo make install

export PATH=$PATH:/home/dev105/devapps/STAR/source


Cat  | xargs Rscript --vanilla pipeline.R 

#Read the sample from the command line
sranumber<-commandArgs(trailingOnly=TRUE) 
print(sranumber)
#Function Idk yet
execute<-function(x, outputfile=NA, intern=FALSE, quitOnError=FALSE){
  if(!is.na(outputfile) &&file.exists(outputfile)){
    cat("Output for step exists, skipping this step\n");
    invisible("")
  }
  cat("----", x, "\n"); res<-system(x, intern=intern); cat(">>>>", res[1], "\n")
  if(res[1] >=1){ 
    cat("Error external process did not finish\n\n");
    if(quitOnError) q("no")
  }
}

input.dir<-"/home/dev105/projects/rawdata"
input.base<-sranumber[1]
output.dir<-paste0("/home/dev105/projects/outputdata/", input.base,".aln")

#Create an output folder
if(!file.exists(input.dir)){ dir.create(input.dir, recursive=TRUE) }
if(!file.exists(output.dir)){ dir.create(output.dir, recursive=TRUE) }

#STEP 0 - SRA Download and Compress
setwd(input.dir)
execute(paste0("prefetch -p", input.base)
execute(paste0("fasterq-dump -p --split-files ", input.base), paste0(input.base, "_1.fastq"))
execute(paste0("bgzip ", input.base, "_1.fastq"), paste0(input.base, "_1.fastq.gz"))
execute(paste0("bgzip ", input.base, "_2.fastq"), paste0(input.base, "_2.fastq.gz"))

#STEP 1 - READ Trimming
trim.files<-c(
                  paste0(input.dir, "/", input.base,"_1.fastq.gz"),
                  paste0(input.dir, "/", input.base,"_2.fastq.gz"),
                  paste0(output.dir, "/", input.base,"_1.P.fastq.gz"),
                  paste0(output.dir, "/", input.base,"_1.U.fastq.gz"),
                  paste0(output.dir, "/", input.base,"_2.P.fastq.gz"),
                  paste0(output.dir, "/", input.base,"_2.U.fastq.gz")
                )
trim.path<-"/home/dev105/devapps/Trimmomatic-0.39"
trim.exec<-paste0("java -jar ", trim.path, "/trimmomatic-0.39.jar")
trim.opts<-paste0("ILLUMINACLIP:",trim.path,"/adapters/TruSeq3-PE-2.fa:2:30:10")
trim.opts<-paste0(trim.opts, "LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36")
trim.cmd<-paste0(trim.exec, "PE ", paste0(trim.files, collapse=""), "", trim.opts)

execute(trim.cmd, trim.files[3])

q("no")



My index for genome star alignment
https://labshare.cshl.edu/shares/gingeraslab/www-data/dobin/STAR/STARgenomes/Human/
https://data.broadinstitute.org/Trinity/
https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/




